In [2]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, ExponentialWeightedMovingAverage, RSI
import pandas as pd

In [3]:
class CloseOnN(CustomFactor):  
    # Define inputs
    inputs = [USEquityPricing.close]
    
    # Set window_length to whatever number of days to lookback as a default
    # in the case where no window_length is given when instantiated.
    # This can also be set/over-ridden as shown below:
    # my_close_on_10 = CloseOnN(window_length = 10)
    
    window_length = 2 
    
    def compute(self, today, assets, out, close):  
        out[:] = close[0]

In [4]:
# We're going to need a date string for some of our queries. Let's grab one.
import datetime

today = datetime.datetime.now().strftime('%Y-%m-%d')
begin = (datetime.datetime.now()-datetime.timedelta(days=5)).strftime('%Y-%m-%d')

In [5]:
# Create pipeline filters on the market cap and p/e ratio
big_market_cap = Fundamentals.market_cap.latest < 100000000000
small_market_cap = Fundamentals.market_cap.latest > 1500000000

# big_pe = Fundamentals.pe_ratio.latest > 5

# Combine those pipeline filters with the QTradableStocksUS, a liquid trading universe
# https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus
universe = QTradableStocksUS() & big_market_cap & small_market_cap

close_11 = CloseOnN(window_length = 11)
close_10 = CloseOnN(window_length = 10)
close_5 = CloseOnN(window_length = 5)

rsi_custom = RSI(  
                            inputs=[USEquityPricing.close],  
                            window_length = 14)

close_1 = CloseOnN(window_length = 1)

mark_cap = Fundamentals.market_cap.latest/1000000000

ch_1 = ((close_11-close_10)/close_10)*100

ch_5days = ((close_5-close_10)/close_5)*100

# Create the pipeline
pipe = Pipeline(
    columns = {
        'close_11' : close_11,
        'close_10' : close_10,
        'close_5' : close_5,
        'initial_change' : ch_1,
        'change_5' : ch_5days,
        'market_cap' : mark_cap,
        'rsi' : rsi_custom,
    },

    screen = universe
)

# Run the pipeline

# start_date = today, end_date = today
fundamental_data = run_pipeline(pipe, '01-01-2014', '03-01-2015')
fundamental_data = fundamental_data.round(2)
fundamental_data = fundamental_data[['close_11', 'close_10', 'initial_change', 'close_5','change_5','market_cap', 'rsi']]
fundamental_data

close_11  close_10  \
2014-01-02 00:00:00+00:00 Equity(2 [ARNC])          9.56      9.71   
                          Equity(52 [ABM])         27.68     27.25   
                          Equity(62 [ABT])         36.54     36.60   
                          Equity(64 [ABX])         17.06     17.19   
                          Equity(67 [ADSK])        47.18     47.50   
                          Equity(76 [TAP])         53.74     53.80   
                          Equity(110 [ACXM])       37.34     37.39   
                          Equity(114 [ADBE])       58.50     57.67   
                          Equity(122 [ADI])        48.73     48.95   
                          Equity(128 [ADM])        40.67     41.47   
                          Equity(154 [AEM])        26.33     26.00   
                          Equity(161 [AEP])        45.96     45.81   
                          Equity(166 [AES])        13.97     13.93   
                          Equity(168 [AET])        65.61     65.75   
                          Equity(185 [AFL])        66.35     66.00   
                          Equity(197 [AGCO])       60.58     57.57   
                          Equity(205 [AGN])       100.24     99.06   
                          Equity(216 [HES])        79.38     79.51   
                          Equity(239 [AIG])        50.29     49.36   
                          Equity(266 [AJG])        45.90     45.54   
                          Equity(270 [AKRX])       24.46     24.20   
                          Equity(300 [ALK])        72.25     70.66   
                          Equity(301 [ALKS])       39.00     38.80   
                          Equity(328 [ALTR])       31.10     31.59   
                          Equity(337 [AMAT])       16.66     16.80   
                          Equity(338 [BEAM])       65.94     66.19   
                          Equity(351 [AMD])         3.59      3.65   
                          Equity(353 [AME])        49.84     50.13   
                          Equity(357 [TWX])        66.31     65.79   
                          Equity(368 [AMGN])      112.17    111.01   
...                                                  ...       ...   
2015-03-02 00:00:00+00:00 Equity(46002 [NMBL])     22.37     23.19   
                          Equity(46015 [ALLY])     21.83     22.04   
                          Equity(46027 [AMC])      30.39     30.62   
                          Equity(46180 [OGS])      42.30     41.66   
                          Equity(46182 [RSPP])     27.20     28.48   
                          Equity(46191 [EPE])      12.34     12.91   
                          Equity(46206 [CPG])      26.03     26.20   
                          Equity(46215 [SC])       21.73     22.11   
                          Equity(46240 [RICE])     19.37     19.16   
                          Equity(46285 [RARE])     53.93     54.16   
                          Equity(46308 [ASPX])     62.95     65.72   
                          Equity(46369 [KN])       21.61     18.59   
                          Equity(46569 [PCTY])     28.35     28.62   
                          Equity(46610 [KING])     13.82     15.72   
                          Equity(46633 [TNET])     35.72     36.49   
                          Equity(46644 [OUT])      29.36     29.60   
                          Equity(46693 [GRUB])     38.20     39.90   
                          Equity(46694 [IMS])      25.46     25.67   
                          Equity(46721 [LQ])       21.58     21.59   
                          Equity(46760 [NYRT])     10.36     10.41   
                          Equity(46764 [FSIC])      9.44      9.50   
                          Equity(46776 [SABR])     21.75     21.73   
                          Equity(46782 [NAVI])     22.21     22.37   
                          Equity(46907 [WPG])      17.05     17.09   
                          Equity(46918 [ZEN])      24.37     25.37   
                          Equity(46929 [TRUE])     19.42  

In [6]:
big_gain = fundamental_data['initial_change'] > 6
price_less_than = fundamental_data['close_11'] < 12
price_greate_than = price_less_than = fundamental_data['close_11'] > 4
fundamental_data = fundamental_data[big_gain & price_less_than]
fundamental_data

close_11  close_10  \
2014-01-02 00:00:00+00:00 Equity(13711 [PCYC])    107.10     99.70   
                          Equity(15129 [FDS])     117.06    109.57   
                          Equity(23120 [OPK])       9.09      8.40   
                          Equity(32871 [AFSI])     35.10     32.59   
2014-01-03 00:00:00+00:00 Equity(2673 [F])         16.69     15.65   
                          Equity(8831 [JBL])       19.72     15.68   
                          Equity(10898 [ALB])      68.80     63.81   
                          Equity(27206 [PAY])      25.00     23.20   
                          Equity(32871 [AFSI])     32.59     28.49   
2014-01-06 00:00:00+00:00 Equity(484 [ATU])        38.45     36.16   
                          Equity(4199 [KBH])       17.55     16.46   
                          Equity(5262 [NEOG])      48.32     43.92   
                          Equity(6330 [RAD])        5.76      5.17   
                          Equity(6961 [SMTC])      28.08     24.69   
                          Equity(28091 [ROSE])     49.26     46.41   
2014-01-07 00:00:00+00:00 Equity(5199 [NAV])       39.46     37.11   
                          Equity(8306 [WOR])       44.03     41.45   
                          Equity(16511 [KMX])      53.05     48.08   
                          Equity(20438 [TIBX])     24.48     21.94   
                          Equity(40606 [KNX])      23.55     22.06   
                          Equity(42611 [NSM])      39.70     37.00   
2014-01-10 00:00:00+00:00 Equity(19831 [BB])        7.74      7.05   
2014-01-14 00:00:00+00:00 Equity(13698 [MYGN])     24.17     20.80   
                          Equity(26994 [DRYS])      4.69      4.39   
2014-01-16 00:00:00+00:00 Equity(24822 [SAM])     241.89    222.16   
                          Equity(26994 [DRYS])      4.70      4.30   
2014-01-21 00:00:00+00:00 Equity(508 [AIRM])       57.83     54.54   
                          Equity(5583 [ODP])        5.25      4.84   
                          Equity(12107 [SSYS])    136.49    127.99   
                          Equity(19187 [CLDX])     24.28     22.33   
...                                                  ...       ...   
2015-02-25 00:00:00+00:00 Equity(15789 [DNR])       8.90      8.15   
                          Equity(24814 [DF])       18.07     15.92   
                          Equity(25707 [WLL])      37.66     35.15   
                          Equity(27993 [LINE])     14.24     13.16   
                          Equity(28077 [ACOR])     39.89     35.87   
                          Equity(28232 [BTE])      19.81     18.51   
                          Equity(39495 [SDRL])     14.34     13.48   
                          Equity(39797 [OAS])      17.94     16.16   
                          Equity(42263 [LPI])      13.15     12.36   
                          Equity(46191 [EPE])      12.75     11.79   
2015-02-26 00:00:00+00:00 Equity(15913 [PAG])      52.19     48.45   
                          Equity(26524 [MNKD])      7.57      7.00   
                          Equity(38989 [AOL])      44.77     40.26   
2015-02-27 00:00:00+00:00 Equity(679 [AXP])        86.00     80.48   
                          Equity(1234 [CAKE])      52.87     49.09   
                          Equity(5629 [OII])       54.19     49.61   
                          Equity(7612 [ANDV])      86.83     81.04   
                          Equity(13905 [NTAP])     39.79     36.89   
                          Equity(20133 [PNRA])    176.48    157.00   
                          Equity(22959 [BG])       91.15     80.42   
                          Equity(25705 [XPER])     41.68     39.10   
                          Equity(27406 [THS])      88.71     81.63   
                          Equity(41098 [CSOD])     35.89     33.01   
                          Equity(45866 [ZU])       19.89     14.50   
2015-03-02 00:00:00+00:00 Equity(7346 [TCO])       79.87     74.70   
                          Equity(25955 [AIZ])      66.12  

In [7]:
high_return = fundamental_data['change_5'] > 10
screened_by_return = fundamental_data[high_return]
screened_by_return

close_11  close_10  \
2014-01-03 00:00:00+00:00 Equity(32871 [AFSI])     32.59     28.49   
2014-01-21 00:00:00+00:00 Equity(19187 [CLDX])     24.28     22.33   
2014-01-28 00:00:00+00:00 Equity(25339 [ISRG])    420.00    393.59   
                          Equity(43721 [SCTY])     66.80     62.49   
2014-01-31 00:00:00+00:00 Equity(16649 [LOGI])     14.78     13.72   
2014-02-07 00:00:00+00:00 Equity(4656 [MTW])       25.06     23.63   
                          Equity(24124 [WYNN])    205.29    192.97   
2014-02-18 00:00:00+00:00 Equity(41730 [ZG])       82.09     77.34   
                          Equity(42689 [PBYI])    118.05    110.28   
2014-02-19 00:00:00+00:00 Equity(4118 [JCP])        5.69      5.09   
                          Equity(16820 [TTWO])     18.90     17.03   
2014-02-20 00:00:00+00:00 Equity(42689 [PBYI])    115.23    106.04   
2014-02-21 00:00:00+00:00 Equity(41579 [P])        35.83     32.23   
2014-02-24 00:00:00+00:00 Equity(34443 [EXXI])     23.10     21.44   
2014-02-26 00:00:00+00:00 Equity(42821 [BLOX])     33.13     17.19   
2014-02-27 00:00:00+00:00 Equity(16307 [VSAT])     59.91     56.13   
2014-03-05 00:00:00+00:00 Equity(43721 [SCTY])     79.95     74.00   
2014-03-06 00:00:00+00:00 Equity(4118 [JCP])        6.00      5.65   
                          Equity(34117 [PPO])      32.99     30.00   
2014-03-10 00:00:00+00:00 Equity(4118 [JCP])        5.66      5.24   
                          Equity(26169 [SHLD])     40.92     38.05   
2014-03-25 00:00:00+00:00 Equity(40294 [AEGR])     52.41     48.81   
2014-04-02 00:00:00+00:00 Equity(16059 [NUS])      76.66     72.23   
2014-04-09 00:00:00+00:00 Equity(36628 [GTAT])     17.49     16.37   
2014-04-10 00:00:00+00:00 Equity(30759 [KS])       28.72     26.25   
2014-04-25 00:00:00+00:00 Equity(42815 [SPLK])     65.04     58.40   
2014-04-28 00:00:00+00:00 Equity(26892 [HLF])      59.84     51.22   
                          Equity(41730 [ZG])       93.65     87.19   
2014-04-29 00:00:00+00:00 Equity(26322 [ACAD])     19.04     17.83   
                          Equity(44965 [GOGO])     18.19     16.80   
...                                                  ...       ...   
2015-01-29 00:00:00+00:00 Equity(14484 [CENX])     22.80     21.03   
                          Equity(27323 [NGD])       4.32      4.00   
                          Equity(31886 [TECK])     12.27     11.51   
2015-01-30 00:00:00+00:00 Equity(19336 [WFT])      10.06      9.46   
                          Equity(28116 [GPOR])     37.36     35.14   
2015-02-03 00:00:00+00:00 Equity(25707 [WLL])      28.29     26.50   
2015-02-04 00:00:00+00:00 Equity(42689 [PBYI])    216.98    191.99   
2015-02-05 00:00:00+00:00 Equity(11999 [AZPN])     34.48     32.28   
2015-02-06 00:00:00+00:00 Equity(8329 [X])         22.72     20.58   
2015-02-10 00:00:00+00:00 Equity(22660 [BTU])       6.67      6.24   
2015-02-11 00:00:00+00:00 Equity(4664 [SM])        38.04     34.77   
                          Equity(5126 [MUR])       46.77     43.40   
                          Equity(5214 [NBR])       11.89     10.48   
                          Equity(5855 [PDS])        5.51      4.81   
                          Equity(15789 [DNR])       7.14      6.46   
                          Equity(17358 [CRZO])     44.73     41.60   
                          Equity(22215 [ERF])       9.90      9.22   
                          Equity(22406 [UPL])      13.63     12.37   
                          Equity(25707 [WLL])      30.76     27.93   
                          Equity(28232 [BTE])      16.09     14.96   
                          Equity(39797 [OAS])      13.45     12.55   
                          Equity(42251 [WPX])      11.48     10.46   
                          Equity(42263 [LPI])      10.05      9.06   
                          Equity(42436 [SLCA])     26.76     24.76   
2015-02-12 00:00:00+00:00 Equity(13508 [CLB])      99.75     89.09   
2015-02-18 00:00:00+00:00 Equity(12107 [SSYS])     79.98  

In [8]:

new = {
       'fund_shape' : fundamental_data.shape,
       'screened_shape': screened_by_return.shape,

       'mean': screened_by_return['rsi'].mean(),
       'median': screened_by_return['rsi'].median(),
       'max': screened_by_return['rsi'].max(),
       'min': screened_by_return['rsi'].min(),



}
new_df = pd.DataFrame(data=new)
new_df

,fund_shape,max,mean,median,min,screened_shape
0,2790,85.55,54.670636,53.92,23.23,173
1,7,85.55,54.670636,53.92,23.23,7
